In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly as py
import pandas as pd
from chart_studio.plotly import plot, iplot

# from plotly.offline import init_notebook_mode, iplot
from tqdm import tqdm_notebook

from scvi.dataset import PbmcDataset
from scvi.models import VAE, IAVAE
from scvi.inference import UnsupervisedTrainer
from scvi.utils import demultiply, make_dir_if_necessary, predict_de_genes, save_fig
from scvi_utils import estimate_de_proba, estimate_lfc_density, estimate_lfc_mean
from R_interop import all_predictions


N_EPOCHS = 100
DELTA = 0.5
# SIZES = [5, 10, 20, 30, 50, 100]
MODE = "cloud"
SIZE = 100
SIZES = [SIZE]
N_SIZES = len(SIZES)

Q0 = 5e-2
N_TRAININGS = 5
N_PICKS = 10

np.random.seed(42)

PATH_TO_SCRIPTS = "/home/ubuntu/conquer_comparison/scripts"
DIR_PATH = 'lfc_estimates/pbmc'
make_dir_if_necessary(DIR_PATH)

label_a = 0
label_b = 4

In [ ]:
import chart_studio.plotly as py

py.sign_in("pierreboyeau", "2wvdnWZ2Qut1zD07ADVy")

# Import Dataset

In [ ]:
dataset = PbmcDataset()

unique_elements, counts_elements = np.unique(
    dataset.labels.squeeze(), return_counts=True
)

df = pd.DataFrame(dict(counts=counts_elements, cell_types=dataset.cell_types))
fig = px.scatter(df, y="counts", x="cell_types")
fig.show()
n_genes = dataset.nb_genes

In [ ]:
print("Cell types: ", dataset.cell_types)
print('Gene names: ', dataset.gene_names)

microarray_info = dataset.de_metadata.set_index('ENSG')
microarray_info = microarray_info.loc[dataset.gene_names]

display(dataset.de_metadata.head())
print(dataset.de_metadata.info())

In [ ]:
n_examples = len(dataset)
labels = dataset.labels.squeeze()
interesting_indices = np.where((labels == label_a) | (labels == label_b))[0]
TEST_INDICES = np.random.permutation(interesting_indices)[:800]
# TEST_INDICES = np.random.permutation(len(dataset))[:3000]

x_test, y_test = dataset.X[TEST_INDICES, :], dataset.labels[TEST_INDICES, :].squeeze()
data_path = os.path.join(DIR_PATH, 'data.npy')
labels_path = os.path.join(DIR_PATH, 'labels.npy')

np.save(
    data_path,
    np.array(x_test.todense()).squeeze().astype(int)
)
np.savetxt(
    labels_path,
    y_test.squeeze()
)

## Train parameters

In [ ]:
mdl_params = dict(
    iaf=dict(n_hidden=128, n_layers=1, do_h=True, n_latent=10, t=4),
    mf=dict(n_hidden=128, n_layers=1, n_latent=10),
    iaf_k5=dict(n_hidden=128, n_layers=1, do_h=True, n_latent=10, t=4),
    mf_k5=dict(n_hidden=128, n_layers=1, n_latent=10),
)
train_params = dict(
    iaf=dict(ratio_loss=True, test_indices=TEST_INDICES),
    mf=dict(ratio_loss=True, test_indices=TEST_INDICES),
    iaf_k5=dict(ratio_loss=True, test_indices=TEST_INDICES, k_importance_weighted=5),
    mf_k5=dict(ratio_loss=True, test_indices=TEST_INDICES, k_importance_weighted=5)
)
train_fn_params = dict(
    iaf=dict(n_epochs=N_EPOCHS, lr=1e-3),
    mf=dict(n_epochs=N_EPOCHS, lr=1e-3),
    iaf_k5=dict(n_epochs=N_EPOCHS, lr=1e-3),
    mf_k5=dict(n_epochs=N_EPOCHS, lr=1e-3),
)

# Competitors


In [ ]:
other_predictions = all_predictions(
    filename=os.path.join(DIR_PATH, "all_predictions.pickle"),
    n_genes=n_genes,
    n_picks=N_PICKS,
    sizes=SIZES,
    data_path=data_path,
    labels_path=labels_path,
    path_to_scripts=PATH_TO_SCRIPTS,
    label_a=label_a,
    label_b=label_b,
    all_nature=True
)


# Experiments

## Microarray

### BDT : size 100

In [ ]:
def bdt_densities(
    filename, mdl_class, dataset, mdl_params, train_params, train_fn_params, sz=SIZE
):
    res = estimate_lfc_density(
        filename=filename,
        mdl_class=mdl_class,
        dataset=dataset,
        mdl_params=mdl_params,
        train_params=train_params,
        train_fn_params=train_fn_params,
        sizes=[sz],
        n_picks=1,
        label_a=0,
        label_b=4,
        n_samples=100
    )[sz].squeeze()
    return res


lfcs_mf = bdt_densities(
    filename=os.path.join(DIR_PATH, "bdt100MF.pickle"),
    mdl_class=VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
)

lfcs_ia = bdt_densities(
    filename=os.path.join(DIR_PATH, "bdt100IAF.pickle"),
    mdl_class=IAVAE,
    dataset=dataset,
    mdl_params=mdl_params["iaf"],
    train_params=train_params["iaf"],
    train_fn_params=train_fn_params["iaf"],
)

# lfcs_iwia = estimate_lfc_density(
#     IAVAE,
#     dataset=dataset,
#     mdl_params=mdl_params["iaf_k5"],
#     train_params=train_params["iaf_k5"],
#     train_fn_params=train_fn_params["iaf_k5"],
#     sizes=[SIZE],
#     n_picks=1,
#     label_a=label_a,
#     label_b=label_b
# )[SIZE].squeeze()

# lfcs_iwmf = estimate_lfc_density(
#     IAVAE,
#     dataset=dataset,
#     mdl_params=mdl_params["mf_k5"],
#     train_params=train_params["mf_k5"],
#     train_fn_params=train_fn_params["mf_k5"],
#     sizes=[SIZE],
#     n_picks=1,
#     label_a=label_a,
#     label_b=label_b
# )[SIZE].squeeze()

In [ ]:
from sklearn.preprocessing import StandardScaler

new_n_genes = 100
std_scaler = StandardScaler(with_mean=False)
std_scaler.fit(dataset.X.astype(np.float64))
subset_genes = np.argsort(std_scaler.var_)[::-1][:new_n_genes]

In [ ]:
from plotly.subplots import make_subplots

lfcs_mf_est = lfcs_ia.reshape((-1, n_genes))[:, subset_genes]
lfcs_ia_est = lfcs_mf.reshape((-1, n_genes))[:, subset_genes]
lfcs_gt = - microarray_info.BDC_logFC[subset_genes]

print(lfcs_mf_est.shape)
print(lfcs_ia_est.shape)
print(lfcs_gt.shape)

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Mean Field", "Inverse Autoregressive Flows"),
    shared_xaxes=True,
    shared_yaxes=True,
)


def add_plot(fig, lfcs_est_m, lfcs_est_err, row, col):
    fig.add_trace(
        go.Scatter(
            x=lfcs_gt,
            y=lfcs_est_m,
            error_y=dict(type="data", array=lfcs_est_err, visible=True),
            mode="markers",
        ),
        row=row,
        col=col,
    )
    return


add_plot(fig, lfcs_mf_est.mean(0), 2.0*lfcs_mf_est.std(0), row=1, col=1)
fig.add_trace(
    go.Scatter(
        x=[-3, 3],
        y=[-3, 3],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=1,
)
add_plot(fig, lfcs_ia_est.mean(0), 2.0*lfcs_ia_est.std(0), row=1, col=2)
fig.add_trace(
    go.Scatter(
        x=[-3, 3],
        y=[-3, 3],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=2,
)

fig.update_xaxes(title_text="Ground Truth LFC", row=1, col=1)
fig.update_xaxes(title_text="Ground Truth LFC", row=2, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=1, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=2, col=1)


fig.update_layout(
    height=600, width=1000, title_text="LFC estimation for {} sample cells B cells/DT cells".format(SIZE)
)
iplot(fig, filename="pbmc_microarray_lfc_with_uncertainty_{}cells_BDT".format(SIZE), sharing="private")

In [ ]:
# import statsmodels.api as sm
# # 'BDC_logFC', y='lfc_MF'
# y = microarray_info.lfc_MF
# X = microarray_info.BDC_logFC

# model = sm.OLS(y, X).fit()

# # Print out the statistics
# model.summary()

### BDT Size 10

In [ ]:
lfcs_mf = bdt_densities(
    filename=os.path.join(DIR_PATH, "bdt10MF.pickle"),
    mdl_class=VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
    sz=10
)

lfcs_ia = bdt_densities(
    filename=os.path.join(DIR_PATH, "bdt10IAFF.pickle"),
    mdl_class=IAVAE,
    dataset=dataset,
    mdl_params=mdl_params["iaf"],
    train_params=train_params["iaf"],
    train_fn_params=train_fn_params["iaf"],
    sz=10
)

In [ ]:
lfcs_mf_est = lfcs_ia.reshape((-1, n_genes))[:, subset_genes]
lfcs_ia_est = lfcs_mf.reshape((-1, n_genes))[:, subset_genes]
lfcs_gt = - microarray_info.BDC_logFC[subset_genes]

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Mean Field", "Inverse Autoregressive Flows"),
    shared_xaxes=True,
    shared_yaxes=True,
)

add_plot(fig, lfcs_mf_est.mean(0), 2.0*lfcs_mf_est.std(0), row=1, col=1)
fig.add_trace(
    go.Scatter(
        x=[-5, 5],
        y=[-5, 5],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=1,
)
add_plot(fig, lfcs_ia_est.mean(0), 2.0*lfcs_ia_est.std(0), row=1, col=2)
fig.add_trace(
    go.Scatter(
        x=[-5, 5],
        y=[-5, 5],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=2,
)

fig.update_xaxes(title_text="Ground Truth LFC", row=1, col=1)
fig.update_xaxes(title_text="Ground Truth LFC", row=2, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=1, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=2, col=1)

fig.update_layout(
    height=600, width=1000, title_text="LFC estimation for {} sample cells B cells/DT cells".format(10)
)

iplot(fig, filename="pbmc_microarray_lfc_with_uncertainty_{}cellsBDT".format(10), sharing="private")

### CD: size 100

In [ ]:
def cd_densities(
    filename, mdl_class, dataset, mdl_params, train_params, train_fn_params, sz=SIZE
):
    res = estimate_lfc_density(
        filename=filename,
        mdl_class=mdl_class,
        dataset=dataset,
        mdl_params=mdl_params,
        train_params=train_params,
        train_fn_params=train_fn_params,
        sizes=[sz],
        n_picks=1,
        label_a=2,
        label_b=3,
        n_samples=100
    )[sz].squeeze()
    return res

In [ ]:
lfcs_mf = cd_densities(
    filename=os.path.join(DIR_PATH, "cd100MF.pickle"),
    mdl_class=VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
)

lfcs_ia = cd_densities(
    filename=os.path.join(DIR_PATH, "cd100IAF.pickle"),
    mdl_class=IAVAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
)


In [ ]:
lfcs_mf_est = lfcs_ia.reshape((-1, n_genes))[:, subset_genes]
lfcs_ia_est = lfcs_mf.reshape((-1, n_genes))[:, subset_genes]
lfcs_gt = - microarray_info.BDC_logFC[subset_genes]

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Mean Field", "Inverse Autoregressive Flows"),
    shared_xaxes=True,
    shared_yaxes=True,
)

add_plot(fig, lfcs_mf_est.mean(0), 2.0*lfcs_mf_est.std(0), row=1, col=1)
fig.add_trace(
    go.Scatter(
        x=[-5, 5],
        y=[-5, 5],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=1,
)
add_plot(fig, lfcs_ia_est.mean(0), 2.0*lfcs_ia_est.std(0), row=1, col=2)
fig.add_trace(
    go.Scatter(
        x=[-5, 5],
        y=[-5, 5],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=2,
)

fig.update_xaxes(title_text="Ground Truth LFC", row=1, col=1)
fig.update_xaxes(title_text="Ground Truth LFC", row=2, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=1, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=2, col=1)

fig.update_layout(
    height=600, width=1000, title_text="LFC estimation for {} sample CD cells".format(SIZE)
)

iplot(fig, filename="pbmc_microarray_lfc_with_uncertainty_{}cellsCD".format(SIZE), sharing="private")

### CD: size 10

In [ ]:
lfcs_mf = cd_densities(
    filename=os.path.join(DIR_PATH, "cd10MF.pickle")
    mdl_class=VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
    sz=10
)

lfcs_ia = cd_densities(
    filename=os.path.join(DIR_PATH, "cd10IAF.pickle")
    mdl_class=IAVAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
    sz=10
)

In [ ]:
lfcs_mf_est = lfcs_ia.reshape((-1, n_genes))[:, subset_genes]
lfcs_ia_est = lfcs_mf.reshape((-1, n_genes))[:, subset_genes]
lfcs_gt = - microarray_info.BDC_logFC[subset_genes]

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Mean Field", "Inverse Autoregressive Flows"),
    shared_xaxes=True,
    shared_yaxes=True,
)

add_plot(fig, lfcs_mf_est.mean(0), 2.0*lfcs_mf_est.std(0), row=1, col=1)
fig.add_trace(
    go.Scatter(
        x=[-5, 5],
        y=[-5, 5],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=1,
)
add_plot(fig, lfcs_ia_est.mean(0), 2.0*lfcs_ia_est.std(0), row=1, col=2)
fig.add_trace(
    go.Scatter(
        x=[-5, 5],
        y=[-5, 5],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    ),
    row=1,
    col=2,
)

fig.update_xaxes(title_text="Ground Truth LFC", row=1, col=1)
fig.update_xaxes(title_text="Ground Truth LFC", row=2, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=1, col=1)
fig.update_yaxes(title_text="Predicted LFC", row=2, col=1)

fig.update_layout(
    height=600, width=1000, title_text="LFC estimation for {} sample CD cells".format(10)
)

iplot(fig, filename="pbmc_microarray_lfc_with_uncertainty_{}cellsCD".format(10), sharing="private")

## Overlap

Voronoi Graph AKA Venn Graph

In [ ]:
de_probas_mf = estimate_de_proba(
    VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
    sizes=[SIZE],
    n_trainings=1,
    n_picks=1,
).squeeze()

In [ ]:
is_pred_de_mf = predict_de_genes(de_probas_mf, desired_fdr=Q0)

## Robustness

As Usual Graph